# Proyecto 1 Inteligencia Artificial: 
## Implementación de redes neuronales para el reconocimiento de voz a partir de espectogramas
### Javier Alonso Rojas Rojas & Brandon Emmanuel Sánchez Araya

# Dataset Raw

In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
# =====================================================
# CONFIGURACIÓN DE RUTAS
# =====================================================
DATASET_PATH = "data/ESC-50-master/audio"
CSV_PATH = "data/ESC-50-master/meta/esc50.csv"
OUTPUT_PATH = "spectrograms"

# =====================================================
# CARGA DE METADATOS
# =====================================================
os.makedirs(OUTPUT_PATH, exist_ok=True)
df = pd.read_csv(CSV_PATH)

# =====================================================
# PROCESAMIENTO DE TODOS LOS AUDIOS
# =====================================================
print(f"Audios encontrados: {len(df)}")
print("Generando espectrogramas...\n")

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Progreso"):
    audio_path = os.path.join(DATASET_PATH, row["filename"])
    categoria = row["category"]

    # Crear carpeta de salida por categoría
    output_dir = os.path.join(OUTPUT_PATH, categoria)
    os.makedirs(output_dir, exist_ok=True)

    try:
        y, sr = librosa.load(audio_path, sr=None)
    except Exception as e:
        print(f"Error al cargar {audio_path}: {e}")
        continue

    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    mel_db = librosa.power_to_db(mel, ref=np.max)

    plt.figure(figsize=(2.24, 2.24), dpi=100)
    plt.axis('off')
    librosa.display.specshow(mel_db, sr=sr, fmax=8000, cmap='gray')  # <-- escala de grises

    output_file = os.path.join(output_dir, row["filename"].replace(".wav", ".png"))
    plt.savefig(output_file, bbox_inches='tight', pad_inches=0)
    plt.close()

print("\nProceso finalizado.")

# Dataset Augmented

In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm

# =====================================================
# CONFIGURACIÓN DE RUTAS
# =====================================================
DATASET_PATH = "ESC-50/audio"
CSV_PATH = "ESC-50/meta/esc50.csv"
OUTPUT_PATH = "spectrograms_augmented"

os.makedirs(OUTPUT_PATH, exist_ok=True)

# =====================================================
# CARGA DE METADATOS
# =====================================================
df = pd.read_csv(CSV_PATH)
print(f"Audios encontrados: {len(df)}")
print("Generando espectrogramas aumentados...\n")

# =====================================================
# PROCESAMIENTO DE TODOS LOS AUDIOS
# =====================================================
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Progreso"):
    audio_path = os.path.join(DATASET_PATH, row["filename"])
    categoria = row["category"]

    # Crear carpeta destino
    output_dir = os.path.join(OUTPUT_PATH, categoria)
    os.makedirs(output_dir, exist_ok=True)

    # --- Cargar audio ---
    try:
        y, sr = librosa.load(audio_path, sr=None)
    except Exception as e:
        print(f"Error al cargar {audio_path}: {e}")
        continue

    # --- AUMENTACIONES INTEGRADAS ---
    # 1. Ruido
    noise_factor = 0.005
    noise = np.random.randn(len(y))
    y_noise = y + noise_factor * noise

    # 2. Time stretch (variar velocidad)
    rate = np.random.uniform(0.9, 1.1)
    try:
        y_stretch = librosa.effects.time_stretch(y, rate=rate)
    except TypeError:
        y_stretch = librosa.effects.time_stretch(y=y, rate=rate)

    # 3. Pitch shift (variar tono)
    n_steps = np.random.randint(-2, 3)
    try:
        y_pitch = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)
    except TypeError:
        y_pitch = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=n_steps)

    # --- CONJUNTO DE VERSIONES ---
    augmentations = {
        "noise": y_noise,
        "stretch": y_stretch,
        "pitch": y_pitch
    }

    # =====================================================
    # GENERAR ESPECTROGRAMAS PARA CADA VERSIÓN
    # =====================================================
    for aug_type, y_aug in augmentations.items():
        mel = librosa.feature.melspectrogram(y=y_aug, sr=sr, n_mels=128, fmax=8000)
        mel_db = librosa.power_to_db(mel, ref=np.max)

        plt.figure(figsize=(2.24, 2.24), dpi=100)
        plt.axis('off')
        librosa.display.specshow(mel_db, sr=sr, fmax=8000, cmap='gray')  # escala de grises

        output_file = os.path.join(output_dir, row["filename"].replace(".wav", f"_{aug_type}.png"))
        plt.savefig(output_file, bbox_inches='tight', pad_inches=0)
        plt.close()

print("\nProceso finalizado.")


# Configuraciones Globales

In [ ]:
import os, random, csv
from glob import glob
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import precision_score, recall_score, f1_score
import wandb

# =====================================================
# CONFIGURACIÓN GLOBAL
# =====================================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

/home/javialroro/miniconda3/envs/ml/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/javialroro/miniconda3/envs/ml/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened b

# Tratamiento del dataset

In [3]:
class SpectrogramDataset(Dataset):
    def __init__(self, root_dirs, img_size=224, grayscale=True):
        self.samples = [] 
        self.class_to_idx, self.idx_to_class = {}, []

        classes = set()
        for r in root_dirs:
            if not os.path.isdir(r): continue
            for d in os.listdir(r):
                if os.path.isdir(os.path.join(r, d)):
                    classes.add(d)
        self.idx_to_class = sorted(list(classes))
        self.class_to_idx = {c: i for i, c in enumerate(self.idx_to_class)}

        for r in root_dirs:
            for c in self.idx_to_class:
                p = os.path.join(r, c)
                if os.path.isdir(p):
                    for fp in glob(os.path.join(p, "*.png")):
                        self.samples.append((fp, self.class_to_idx[c]))

        self.img_size = img_size
        self.grayscale = grayscale

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path)
        img = img.convert("L") if self.grayscale else img.convert("RGB")
        img = img.resize((self.img_size, self.img_size))
        arr = np.array(img, dtype=np.float32) / 255.0
        if self.grayscale:
            arr = arr[None, :, :]
        else:
            arr = arr.transpose(2, 0, 1)
        x = torch.from_numpy(arr)
        y = torch.tensor(label, dtype=torch.long)
        return x, y

# Modelo A

In [ ]:
class LeNet5(nn.Module):
    def __init__(self, in_channels=1, num_classes=50):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 6, 5),  
            nn.Tanh(),
            nn.MaxPool2d(2, 2),           
            nn.Conv2d(6, 16, 5),
            nn.Tanh(),
            nn.MaxPool2d(2, 2)
        )

        with torch.no_grad():
            dummy = torch.zeros(1, in_channels, 224, 224)
            flat_dim = self.features(dummy).numel()

        self.classifier = nn.Sequential(
            nn.Linear(flat_dim, 120),
            nn.Tanh(),
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  
        return self.classifier(x)

## Entrenamiento

In [ ]:
def run_experiment(dataset_name, data_dirs, hparams):
    print(f"\n=== Dataset: {dataset_name} ===")

    for i, hp in enumerate(hparams, 1):
        print(f"\nEjecutando experimento {i}/{len(hparams)} ...")

        # Inicializa experimento en Weights & Biases
        wandb.init(project="Proyecto1_IA", name=f"{dataset_name}_run{i}")
        wandb.config.update(hp)

        # Carga dataset y divide en Train / Val / Test
        full = SpectrogramDataset(data_dirs, img_size=224, grayscale=True)
        num_classes = len(full.idx_to_class)
        N = len(full)
        n_test = int(0.1 * N)
        n_val = int(0.1 * N)
        n_train = N - n_val - n_test
        train, val, test = random_split(full, [n_train, n_val, n_test],
                                        generator=torch.Generator().manual_seed(SEED))
        train_loader = DataLoader(train, batch_size=hp["batch"], shuffle=True)
        val_loader = DataLoader(val, batch_size=hp["batch"], shuffle=False)
        test_loader = DataLoader(test, batch_size=hp["batch"], shuffle=False)

        # Instancia modelo, pérdida, optimizador y scheduler
        model = LeNet5(in_channels=1, num_classes=num_classes).to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=hp["lr"], weight_decay=hp["wd"])
        scheduler = CosineAnnealingLR(optimizer, T_max=hp["epochs"])

        # Entrenamiento con early saving del mejor modelo
        best_val, best_weights = 0, None

        for epoch in range(1, hp["epochs"] + 1):
            # ---------------- ENTRENAMIENTO ----------------
            model.train()
            total_loss, total_acc, n = 0, 0, 0
            for x, y in train_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                out = model(x)
                loss = criterion(out, y)
                loss.backward()
                optimizer.step()

                # Calcular accuracy directamente
                preds = out.argmax(1)
                acc = (preds == y).float().mean().item()

                bs = y.size(0)
                total_loss += loss.item() * bs
                total_acc += acc * bs
                n += bs

            train_loss = total_loss / n
            train_acc = total_acc / n

            # ---------------- VALIDACIÓN ----------------
            model.eval()
            val_total_loss, val_total_acc, n_val_batches = 0, 0, 0
            preds_all, labels_all = [], []
            with torch.no_grad():
                for x, y in val_loader:
                    x, y = x.to(DEVICE), y.to(DEVICE)
                    out = model(x)
                    loss = criterion(out, y)

                    preds = out.argmax(1)
                    acc = (preds == y).float().mean().item()

                    bs = y.size(0)
                    val_total_loss += loss.item() * bs
                    val_total_acc += acc * bs
                    preds_all.extend(preds.cpu().numpy())
                    labels_all.extend(y.cpu().numpy())
                    n_val_batches += bs

            val_loss = val_total_loss / n_val_batches
            val_acc = val_total_acc / n_val_batches

            scheduler.step()

            f1 = f1_score(labels_all, preds_all, average="macro", zero_division=0)
            prec = precision_score(labels_all, preds_all, average="macro", zero_division=0)
            rec = recall_score(labels_all, preds_all, average="macro", zero_division=0)

            # Registro en WandB
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "val_loss": val_loss,
                "train_acc": train_acc,
                "val_acc": val_acc,
                "f1": f1,
                "precision": prec,
                "recall": rec
            })

            print(f"[{epoch:02d}/{hp['epochs']}] Training Loss={train_loss:.4f} Training Acc={train_acc:.4f} | Validation Loss={val_loss:.4f} Validation Acc={val_acc:.4f} | F1={f1:.4f} Precision={prec:.4f} Recall={rec:.4f}")

            if val_acc > best_val:
                best_val = val_acc
                best_weights = model.state_dict().copy()

        # ---------------- TEST FINAL ----------------
        model.load_state_dict(best_weights)
        model.eval()
        test_total_loss, test_total_acc, n_test_batches = 0, 0, 0
        preds_all, labels_all = [], []

        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                out = model(x)
                loss = criterion(out, y)

                preds = out.argmax(1)
                acc = (preds == y).float().mean().item()

                bs = y.size(0)
                test_total_loss += loss.item() * bs
                test_total_acc += acc * bs
                preds_all.extend(preds.cpu().numpy())
                labels_all.extend(y.cpu().numpy())
                n_test_batches += bs

        test_loss = test_total_loss / n_test_batches
        test_acc = test_total_acc / n_test_batches


        print(f"→ Mejor val_acc={best_val:.4f} | Test_acc={test_acc:.4f} | F1={f1:.4f}")

        wandb.log({
            "test_acc": test_acc,
            "f1": f1,
            "precision": prec,
            "recall": rec
        })
        wandb.finish()

In [5]:
print(f"Usando dispositivo: {DEVICE}")

experimentos = [
    {"lr":1e-3, "wd":1e-4, "batch":32, "epochs":15},
    {"lr":5e-4, "wd":1e-4, "batch":64, "epochs":15},
    {"lr":1e-3, "wd":1e-5, "batch":64, "epochs":20},
    {"lr":5e-4, "wd":1e-3, "batch":32, "epochs":20},
    {"lr":1e-4, "wd":1e-4, "batch":64, "epochs":25},
]

# Entrenamientos: dataset base y dataset aumentado
run_experiment("base", ["spectrograms"], experimentos)
run_experiment("augmented", ["spectrograms_augmented"], experimentos)


Usando dispositivo: cuda

=== Dataset: base ===

Ejecutando experimento 1/5 ...


wandb: Currently logged in as: javialroro (javialroro-tecnologico-de-costa-rica) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[01/15] Training Loss=3.7872 Training Acc=0.0488 | Validation Loss=3.6646 Validation Acc=0.0650 | F1=0.0292 Precision=0.0196 Recall=0.0777
[02/15] Training Loss=3.5576 Training Acc=0.0781 | Validation Loss=3.5791 Validation Acc=0.1050 | F1=0.0677 Precision=0.0842 Recall=0.1260
[03/15] Training Loss=3.3990 Training Acc=0.1125 | Validation Loss=3.4219 Validation Acc=0.1200 | F1=0.0851 Precision=0.0883 Recall=0.1463
[04/15] Training Loss=3.2593 Training Acc=0.1487 | Validation Loss=3.3879 Validation Acc=0.1400 | F1=0.1049 Precision=0.0962 Recall=0.1729
[05/15] Training Loss=3.1570 Training Acc=0.1500 | Validation Loss=3.2397 Validation Acc=0.1650 | F1=0.1163 Precision=0.1045 Recall=0.1787
[06/15] Training Loss=3.0392 Training Acc=0.1975 | Validation Loss=3.1816 Validation Acc=0.1350 | F1=0.1038 Precision=0.0960 Recall=0.1721
[07/15] Training Loss=2.9688 Training Acc=0.2025 | Validation Loss=3.1435 Validation Acc=0.2150 | F1=0.1652 Precision=0.1621 Recall=0.2349
[08/15] Training Loss=2.870

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
f1,▁▃▃▄▄▄▆▅▇▇██████
precision,▁▄▄▄▄▄▇▅▇▇███▇▇▇
recall,▁▃▄▅▅▅▇▆▇███████
test_acc,▁
train_acc,▁▂▂▃▃▄▄▅▆▆▆▇▇██
train_loss,█▇▆▅▅▄▄▃▃▂▂▂▁▁▁
val_acc,▁▂▃▄▄▃▆▅▇▇█████
val_loss,█▇▆▆▄▄▄▃▂▂▂▁▁▁▁
epoch,15
f1,0.20096



Ejecutando experimento 2/5 ...


...

[01/20] Training Loss=3.6274 Training Acc=0.0875 | Validation Loss=3.3139 Validation Acc=0.1733 | F1=0.1067 Precision=0.1358 Recall=0.1709
[02/20] Training Loss=3.0654 Training Acc=0.2240 | Validation Loss=2.9423 Validation Acc=0.2583 | F1=0.2056 Precision=0.2772 Recall=0.2529
[03/20] Training Loss=2.6922 Training Acc=0.3219 | Validation Loss=2.6209 Validation Acc=0.3567 | F1=0.3171 Precision=0.3604 Recall=0.3551
[04/20] Training Loss=2.3321 Training Acc=0.4304 | Validation Loss=2.2798 Validation Acc=0.4450 | F1=0.4065 Precision=0.4358 Recall=0.4379
[05/20] Training Loss=1.8933 Training Acc=0.5740 | Validation Loss=1.9300 Validation Acc=0.5533 | F1=0.5265 Precision=0.5518 Recall=0.5533
[06/20] Training Loss=1.4908 Training Acc=0.7013 | Validation Loss=1.6561 Validation Acc=0.6367 | F1=0.6098 Precision=0.6454 Recall=0.6338
[07/20] Training Loss=1.1065 Training Acc=0.8129 | Validation Loss=1.3801 Validation Acc=0.7150 | F1=0.6946 Precision=0.7315 Recall=0.7089
[08/20] Training Loss=0.787

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
f1,▁▂▃▄▅▆▆▇▇▇███████████
precision,▁▂▃▄▅▆▇▇▇████████████
recall,▁▂▃▄▅▆▆▇▇▇███████████
test_acc,▁
train_acc,▁▂▃▄▅▆▇▇████████████
train_loss,█▇▆▅▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁
val_acc,▁▂▃▄▅▆▆▇▇▇██████████
val_loss,█▇▆▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch,20
f1,0.85926



Ejecutando experimento 5/5 ...


[01/25] Training Loss=3.7605 Training Acc=0.0729 | Validation Loss=3.5908 Validation Acc=0.1317 | F1=0.0734 Precision=0.0721 Recall=0.1289
[02/25] Training Loss=3.4852 Training Acc=0.1704 | Validation Loss=3.4051 Validation Acc=0.1883 | F1=0.1261 Precision=0.1680 Recall=0.1782
[03/25] Training Loss=3.3006 Training Acc=0.2283 | Validation Loss=3.2583 Validation Acc=0.2633 | F1=0.2137 Precision=0.2615 Recall=0.2552
[04/25] Training Loss=3.1389 Training Acc=0.2921 | Validation Loss=3.1324 Validation Acc=0.2800 | F1=0.2229 Precision=0.3040 Recall=0.2716
[05/25] Training Loss=2.9931 Training Acc=0.3471 | Validation Loss=3.0139 Validation Acc=0.3333 | F1=0.2853 Precision=0.3426 Recall=0.3317
[06/25] Training Loss=2.8652 Training Acc=0.3940 | Validation Loss=2.9129 Validation Acc=0.3667 | F1=0.3132 Precision=0.3596 Recall=0.3675
[07/25] Training Loss=2.7402 Training Acc=0.4375 | Validation Loss=2.8122 Validation Acc=0.3933 | F1=0.3412 Precision=0.4282 Recall=0.3905
[08/25] Training Loss=2.628

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
f1,▁▂▃▃▄▅▅▆▆▆▇▇▇▇▇███████████
precision,▁▂▄▄▅▅▆▆▆▆▇▇▇▇▇███████████
recall,▁▂▃▃▄▅▅▆▆▆▇▇▇▇▇███████████
test_acc,▁
train_acc,▁▂▃▃▄▅▅▅▆▆▆▇▇▇▇▇█████████
train_loss,█▇▆▆▅▅▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁
val_acc,▁▂▃▄▄▅▆▆▆▆▇▇▇▇▇██████████
val_loss,█▇▆▆▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch,25
f1,0.49803


# Modelo A con Dropout

In [4]:
class LeNet5_Dropout(nn.Module):
    def __init__(self, in_channels=1, num_classes=50):
        super().__init__()

        # Bloques convolucionales: extracción de características
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 6, 5),  
            nn.Tanh(),
            nn.MaxPool2d(2, 2),            
            nn.Conv2d(6, 16, 5),           
            nn.Tanh(),
            nn.MaxPool2d(2, 2)
        )

        # Calcula automáticamente el tamaño plano de salida
        with torch.no_grad():
            dummy = torch.zeros(1, in_channels, 224, 224)
            flat_dim = self.features(dummy).numel()

        # Capas completamente conectadas (clasificador)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(flat_dim, 120),
            nn.Tanh(),
            nn.Linear(120, 84),
            nn.Dropout(p=0.2),
            nn.Tanh(),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  
        return self.classifier(x)

## Entrenamiento

In [ ]:
def run_experimentAD(dataset_name, data_dirs, hparams):
    print(f"\n=== Dataset: {dataset_name} ===")

    for i, hp in enumerate(hparams, 1):
        print(f"\nEjecutando experimento {i}/{len(hparams)} ...")

        # Inicializa experimento en Weights & Biases
        wandb.init(project="Proyecto1_IA_Dropout", name=f"{dataset_name}_run{i}")
        wandb.config.update(hp)

        # Carga dataset y divide en Train / Val / Test
        full = SpectrogramDataset(data_dirs, img_size=224, grayscale=True)
        num_classes = len(full.idx_to_class)
        N = len(full)
        n_test = int(0.1 * N)
        n_val = int(0.1 * N)
        n_train = N - n_val - n_test
        train, val, test = random_split(full, [n_train, n_val, n_test],
                                        generator=torch.Generator().manual_seed(SEED))
        train_loader = DataLoader(train, batch_size=hp["batch"], shuffle=True)
        val_loader = DataLoader(val, batch_size=hp["batch"], shuffle=False)
        test_loader = DataLoader(test, batch_size=hp["batch"], shuffle=False)

        # Instancia modelo, pérdida, optimizador y scheduler
        model = LeNet5_Dropout(in_channels=1, num_classes=num_classes).to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=hp["lr"], weight_decay=hp["wd"])
        scheduler = CosineAnnealingLR(optimizer, T_max=hp["epochs"])

        # Entrenamiento con early saving del mejor modelo
        best_val, best_weights = 0, None

        for epoch in range(1, hp["epochs"] + 1):
            # ---------------- ENTRENAMIENTO ----------------
            model.train()
            total_loss, total_acc, n = 0, 0, 0
            for x, y in train_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                out = model(x)
                loss = criterion(out, y)
                loss.backward()
                optimizer.step()

                # Calcular accuracy directamente
                preds = out.argmax(1)
                acc = (preds == y).float().mean().item()

                bs = y.size(0)
                total_loss += loss.item() * bs
                total_acc += acc * bs
                n += bs

            train_loss = total_loss / n
            train_acc = total_acc / n

            # ---------------- VALIDACIÓN ----------------
            model.eval()
            val_total_loss, val_total_acc, n_val_batches = 0, 0, 0
            preds_all, labels_all = [], []
            with torch.no_grad():
                for x, y in val_loader:
                    x, y = x.to(DEVICE), y.to(DEVICE)
                    out = model(x)
                    loss = criterion(out, y)

                    preds = out.argmax(1)
                    acc = (preds == y).float().mean().item()

                    bs = y.size(0)
                    val_total_loss += loss.item() * bs
                    val_total_acc += acc * bs
                    preds_all.extend(preds.cpu().numpy())
                    labels_all.extend(y.cpu().numpy())
                    n_val_batches += bs

            val_loss = val_total_loss / n_val_batches
            val_acc = val_total_acc / n_val_batches

            scheduler.step()

            f1 = f1_score(labels_all, preds_all, average="macro", zero_division=0)
            prec = precision_score(labels_all, preds_all, average="macro", zero_division=0)
            rec = recall_score(labels_all, preds_all, average="macro", zero_division=0)

            # Registro en WandB
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "val_loss": val_loss,
                "train_acc": train_acc,
                "val_acc": val_acc,
                "f1": f1,
                "precision": prec,
                "recall": rec
            })

            print(f"[{epoch:02d}/{hp['epochs']}] Training Loss={train_loss:.4f} Training Acc={train_acc:.4f} | Validation Loss={val_loss:.4f} Validation Acc={val_acc:.4f} | F1={f1:.4f} Precision={prec:.4f} Recall={rec:.4f}")

            if val_acc > best_val:
                best_val = val_acc
                best_weights = model.state_dict().copy()

        # ---------------- TEST FINAL ----------------
        model.load_state_dict(best_weights)
        model.eval()
        test_total_loss, test_total_acc, n_test_batches = 0, 0, 0
        preds_all, labels_all = [], []

        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                out = model(x)
                loss = criterion(out, y)

                preds = out.argmax(1)
                acc = (preds == y).float().mean().item()

                bs = y.size(0)
                test_total_loss += loss.item() * bs
                test_total_acc += acc * bs
                preds_all.extend(preds.cpu().numpy())
                labels_all.extend(y.cpu().numpy())
                n_test_batches += bs

        test_loss = test_total_loss / n_test_batches
        test_acc = test_total_acc / n_test_batches


        print(f"→ Mejor val_acc={best_val:.4f} | Test_acc={test_acc:.4f} | F1={f1:.4f}")

        wandb.log({
            "test_acc": test_acc,
            "f1": f1,
            "precision": prec,
            "recall": rec
        })
        wandb.finish()

In [8]:
print(f"Usando dispositivo: {DEVICE}")

experimentos = [
    {"lr":1e-3, "wd":1e-4, "batch":32, "epochs":15},
    {"lr":5e-4, "wd":1e-4, "batch":64, "epochs":15},
    {"lr":1e-3, "wd":1e-5, "batch":64, "epochs":20},
    {"lr":5e-4, "wd":1e-3, "batch":32, "epochs":20},
    {"lr":1e-4, "wd":1e-4, "batch":64, "epochs":25},
]

# Entrenamientos: dataset base y dataset aumentado
run_experimentAD("base", ["spectrograms"], experimentos)
run_experimentAD("augmented", ["spectrograms_augmented"], experimentos)

Usando dispositivo: cuda

=== Dataset: base ===

Ejecutando experimento 1/5 ...


wandb: Currently logged in as: javialroro (javialroro-tecnologico-de-costa-rica) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[01/15] Training Loss=3.8796 Training Acc=0.0325 | Validation Loss=3.7689 Validation Acc=0.0500 | F1=0.0210 Precision=0.0205 Recall=0.0690
[02/15] Training Loss=3.6759 Training Acc=0.0606 | Validation Loss=3.6530 Validation Acc=0.1000 | F1=0.0493 Precision=0.0639 Recall=0.1257
[03/15] Training Loss=3.5573 Training Acc=0.0781 | Validation Loss=3.5824 Validation Acc=0.1100 | F1=0.0580 Precision=0.0695 Recall=0.1457
[04/15] Training Loss=3.4681 Training Acc=0.1006 | Validation Loss=3.5579 Validation Acc=0.0850 | F1=0.0652 Precision=0.0864 Recall=0.1140
[05/15] Training Loss=3.3992 Training Acc=0.1094 | Validation Loss=3.4779 Validation Acc=0.0900 | F1=0.0479 Precision=0.0384 Recall=0.1167
[06/15] Training Loss=3.3197 Training Acc=0.1200 | Validation Loss=3.4290 Validation Acc=0.1150 | F1=0.0844 Precision=0.0867 Recall=0.1392
[07/15] Training Loss=3.2604 Training Acc=0.1356 | Validation Loss=3.3618 Validation Acc=0.1250 | F1=0.0869 Precision=0.1042 Recall=0.1564
[08/15] Training Loss=3.141

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
f1,▁▂▃▃▂▄▄▆▇▇▇█████
precision,▁▄▄▅▂▅▆█▇█▇█▇███
recall,▁▃▄▃▃▄▄▅▆▆▆██▇▇▇
test_acc,▁
train_acc,▁▂▂▃▃▃▄▅▅▆▆▇███
train_loss,█▇▆▅▅▅▄▃▃▂▂▂▁▁▁
val_acc,▁▃▃▂▃▄▄▅▆▆▇████
val_loss,█▇▆▆▅▅▄▃▂▂▂▁▁▁▁
epoch,15
f1,0.15453



Ejecutando experimento 2/5 ...


...

[01/20] Training Loss=3.6800 Training Acc=0.0737 | Validation Loss=3.3620 Validation Acc=0.1750 | F1=0.0996 Precision=0.1142 Recall=0.1642
[02/20] Training Loss=3.1944 Training Acc=0.1840 | Validation Loss=3.0703 Validation Acc=0.2150 | F1=0.1731 Precision=0.2600 Recall=0.2112
[03/20] Training Loss=2.8752 Training Acc=0.2685 | Validation Loss=2.7262 Validation Acc=0.3017 | F1=0.2414 Precision=0.2734 Recall=0.2960
[04/20] Training Loss=2.5695 Training Acc=0.3638 | Validation Loss=2.4313 Validation Acc=0.3850 | F1=0.3416 Precision=0.3879 Recall=0.3860
[05/20] Training Loss=2.1985 Training Acc=0.4873 | Validation Loss=2.1097 Validation Acc=0.4833 | F1=0.4505 Precision=0.4776 Recall=0.4869
[06/20] Training Loss=1.8473 Training Acc=0.6106 | Validation Loss=1.8221 Validation Acc=0.5650 | F1=0.5326 Precision=0.5548 Recall=0.5635
[07/20] Training Loss=1.4990 Training Acc=0.7179 | Validation Loss=1.5976 Validation Acc=0.6267 | F1=0.6021 Precision=0.6279 Recall=0.6181
[08/20] Training Loss=1.198

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
f1,▁▂▂▃▄▅▆▆▇▇▇██████████
precision,▁▂▂▄▄▅▆▇▇▇███████████
recall,▁▁▂▃▄▅▆▆▇▇▇██████████
test_acc,▁
train_acc,▁▂▂▃▄▅▆▇▇▇██████████
train_loss,█▇▆▆▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁
val_acc,▁▁▂▃▄▅▆▆▇▇▇█████████
val_loss,█▇▆▆▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁
epoch,20
f1,0.85887



Ejecutando experimento 5/5 ...


[01/25] Training Loss=3.7853 Training Acc=0.0612 | Validation Loss=3.6150 Validation Acc=0.1267 | F1=0.0702 Precision=0.0859 Recall=0.1212
[02/25] Training Loss=3.5468 Training Acc=0.1460 | Validation Loss=3.4385 Validation Acc=0.1717 | F1=0.1095 Precision=0.1232 Recall=0.1615
[03/25] Training Loss=3.3898 Training Acc=0.2002 | Validation Loss=3.2986 Validation Acc=0.2333 | F1=0.1883 Precision=0.2883 Recall=0.2272
[04/25] Training Loss=3.2535 Training Acc=0.2421 | Validation Loss=3.1841 Validation Acc=0.2583 | F1=0.2060 Precision=0.2888 Recall=0.2555
[05/25] Training Loss=3.1337 Training Acc=0.2998 | Validation Loss=3.0691 Validation Acc=0.3183 | F1=0.2786 Precision=0.3978 Recall=0.3208
[06/25] Training Loss=3.0253 Training Acc=0.3427 | Validation Loss=2.9721 Validation Acc=0.3550 | F1=0.3083 Precision=0.3917 Recall=0.3583
[07/25] Training Loss=2.9183 Training Acc=0.3792 | Validation Loss=2.8806 Validation Acc=0.3817 | F1=0.3343 Precision=0.4482 Recall=0.3820
[08/25] Training Loss=2.822

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
f1,▁▂▃▃▅▅▅▆▆▆▆▆▆▇▇▇▇█████████
precision,▁▂▄▄▆▆▇▇▇▇▇▇▇▇▇▇██████████
recall,▁▂▃▃▄▅▆▆▆▆▇▇▇▇▇▇██████████
test_acc,▁
train_acc,▁▂▃▃▄▄▅▅▆▆▆▇▇▇▇▇▇████████
train_loss,█▇▆▆▅▅▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁
val_acc,▁▂▃▃▄▅▅▆▆▆▇▇▇▇▇▇█████████
val_loss,█▇▆▆▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch,25
f1,0.48446


# Modelo B

In [3]:
from typing import Callable, Optional, Type, List
import torch
import torch.nn as nn


# -------------------------
# Utilidades de convolución
# -------------------------
def conv3x3(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """Conv 3×3 con padding=1, sin bias (BN lo compensa)."""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """Conv 1×1 para proyección en atajos (ajustar canales/stride)."""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


# -------------------------
# Bloque residual "básico"
# -------------------------
class BasicBlock(nn.Module):
    """
    Estructura:
        Conv3x3 → BN → ReLU → Conv3x3 → BN → (Suma con atajo) → ReLU
    Donde el atajo (identity) puede incluir una proyección 1×1 si cambia
    la resolución (stride > 1) o el número de canales.
    """
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1   = norm_layer(planes)
        self.relu  = nn.ReLU(inplace=True)

        self.conv2 = conv3x3(planes, planes)
        self.bn2   = norm_layer(planes)

        self.downsample = downsample  # Proyección para el atajo, si aplica

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x  # Atajo

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Alinear dimensiones del atajo si cambió stride o # de canales
        if self.downsample is not None:
            identity = self.downsample(x)

        out = out + identity
        out = self.relu(out)
        return out


# -----------
# ResNet base
# -----------
class ResNet(nn.Module):
    """
    Constructor general de ResNet con BasicBlock y configuración [2,2,2,2].
    Parámetros clave:
        - small_input=True: conv1=3×3 s=1 y sin MaxPool (mejor para 64–224 px).
        - small_input=False: conv1=7×7 s=2 + MaxPool (clásico de ResNet).
    """
    def __init__(
        self,
        block: Type[BasicBlock],
        layers: List[int],
        num_classes: int = 50,
        in_channels: int = 1,
        small_input: bool = True,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64

        # Capa inicial: variante "small_input" recomendada para espectrogramas
        if small_input:
            # Preserva más detalle inicial (sin MaxPool temprano)
            self.conv1   = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1, bias=False)
            self.maxpool = nn.Identity()
        else:
            # Estilo ResNet clásico para entradas grandes
            self.conv1   = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
            self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.bn1  = norm_layer(64)
        self.relu = nn.ReLU(inplace=True)

        # Stages: [64, 128, 256, 512] con [2, 2, 2, 2] bloques
        self.layer1 = self._make_layer(block,  64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        # Cabeza de clasificación
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Global Average Pooling
        self.fc      = nn.Linear(512 * block.expansion, num_classes)

        # Inicialización recomendada para ReLU/BN
        self._init_weights()

    def _make_layer(self, block: Type[BasicBlock], planes: int, blocks: int, stride: int = 1) -> nn.Sequential:
        """
        Crea un stage con 'blocks' bloques. El primer bloque puede hacer downsample
        (stride=2) para reducir resolución y duplicar canales.
        """
        norm_layer = self._norm_layer
        downsample = None

        # Si cambia resolución o # de canales, proyectamos el atajo (1×1)
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _init_weights(self) -> None:
        """Inicialización Kaiming para conv; constantes para BN; normal para FC."""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1.0)
                nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0.0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Entrada → conv1 → BN → ReLU → (posible MaxPool/Identity)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # Stages residuales
        x = self.layer1(x)  # 64
        x = self.layer2(x)  # 128
        x = self.layer3(x)  # 256
        x = self.layer4(x)  # 512

        # Cabeza
        x = self.avgpool(x)           # (B, 512, 1, 1)
        x = torch.flatten(x, 1)       # (B, 512)
        x = self.fc(x)                # (B, num_classes)
        return x


# -------------------------
# Fábrica de ResNet-18
# -------------------------
def resnet18_audio(num_classes: int = 50, in_channels: int = 1, small_input: bool = True) -> ResNet:
    """
    Retorna una ResNet-18 lista para espectrogramas:
        - num_classes: # de clases del dataset (ESC-50 → 50)
        - in_channels: 1 para grises; 3 si usas RGB (replicar canal)
        - small_input: True recomendado para ~128–224 px
    """
    return ResNet(
        block=BasicBlock,
        layers=[2, 2, 2, 2],
        num_classes=num_classes,
        in_channels=in_channels,
        small_input=small_input,
    )

## Entrenamiento

In [ ]:
def run_experiment(dataset_name, data_dirs, hparams):
    print(f"\n=== Dataset: {dataset_name} ===")

    for i, hp in enumerate(hparams, 1):
        print(f"\nEjecutando experimento {i}/{len(hparams)} ...")

        wandb.init(project="Proyecto1_IA_ResNet", name=f"{dataset_name}_run{i}")
        wandb.config.update(hp)

        # --- Dataset ---
        full = SpectrogramDataset(data_dirs, img_size=224, grayscale=True)
        num_classes = len(full.idx_to_class)
        N = len(full)
        n_test = int(0.1 * N)
        n_val  = int(0.1 * N)
        n_train = N - n_val - n_test
        train, val, test = random_split(full, [n_train, n_val, n_test],
                                        generator=torch.Generator().manual_seed(SEED))
        train_loader = DataLoader(train, batch_size=hp["batch"], shuffle=True, num_workers=2, pin_memory=True)
        val_loader   = DataLoader(val, batch_size=hp["batch"], shuffle=False, num_workers=2, pin_memory=True)
        test_loader  = DataLoader(test, batch_size=hp["batch"], shuffle=False, num_workers=2, pin_memory=True)

        # --- Modelo y optimización ---
        model = resnet18_audio(num_classes=num_classes, in_channels=1).to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=hp["lr"], weight_decay=hp["wd"])
        scheduler = CosineAnnealingLR(optimizer, T_max=hp["epochs"])

        best_val, best_weights = 0, None

        scaler= torch.amp.GradScaler()

        # --- Bucle de entrenamiento integrado ---
        for epoch in range(1, hp["epochs"]+1):
            model.train()
            total_loss, total_acc, n = 0, 0, 0
            for x, y in train_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                with torch.amp.autocast('cuda'):
                    out = model(x)
                    loss = criterion(out, y)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                preds = out.argmax(1)
                acc = (preds == y).float().mean().item()
                bs = y.size(0)
                total_loss += loss.item() * bs
                total_acc += acc * bs
                n += bs
            train_loss = total_loss / n
            train_acc  = total_acc / n

            # --- Validación ---
            model.eval()
            val_loss_sum, val_acc_sum, n_val = 0, 0, 0
            preds_all, labels_all = [], []
            with torch.no_grad():
                for x, y in val_loader:
                    x, y = x.to(DEVICE), y.to(DEVICE)
                    out = model(x)
                    loss = criterion(out, y)
                    preds = out.argmax(1)
                    acc = (preds == y).float().mean().item()
                    bs = y.size(0)
                    val_loss_sum += loss.item() * bs
                    val_acc_sum  += acc * bs
                    preds_all.extend(preds.cpu().numpy())
                    labels_all.extend(y.cpu().numpy())
                    n_val += bs
            val_loss = val_loss_sum / n_val
            val_acc  = val_acc_sum / n_val
            scheduler.step()

            f1   = f1_score(labels_all, preds_all, average="macro", zero_division=0)
            prec = precision_score(labels_all, preds_all, average="macro", zero_division=0)
            rec  = recall_score(labels_all, preds_all, average="macro", zero_division=0)


            # Registro en WandB
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "val_loss": val_loss,
                "train_acc": train_acc,
                "val_acc": val_acc,
                "f1": f1,
                "precision": prec,
                "recall": rec
            })

            print(f"[{epoch:02d}/{hp['epochs']}] Training Loss={train_loss:.4f} Training Acc={train_acc:.4f} | Validation Loss={val_loss:.4f} Validation Acc={val_acc:.4f} | F1={f1:.4f} Precision={prec:.4f} Recall={rec:.4f}")

            if val_acc > best_val:
                best_val = val_acc
                best_weights = model.state_dict().copy()

        # --- Evaluación final (Test) ---
        model.load_state_dict(best_weights)
        model.eval()
        test_loss_sum, test_acc_sum, n_test = 0, 0, 0
        preds_all, labels_all = [], []
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                out = model(x)
                loss = criterion(out, y)
                preds = out.argmax(1)
                acc = (preds == y).float().mean().item()
                bs = y.size(0)
                test_loss_sum += loss.item() * bs
                test_acc_sum  += acc * bs
                preds_all.extend(preds.cpu().numpy())
                labels_all.extend(y.cpu().numpy())
                n_test += bs
        test_loss = test_loss_sum / n_test
        test_acc  = test_acc_sum / n_test

        f1   = f1_score(labels_all, preds_all, average="macro")
        prec = precision_score(labels_all, preds_all, average="macro")
        rec  = recall_score(labels_all, preds_all, average="macro")

        print(f"→ Mejor val_acc={best_val:.4f} | Test_acc={test_acc:.4f} | F1={f1:.4f}")

        wandb.log({"test_acc": test_acc, "f1": f1, "precision": prec, "recall": rec})
        wandb.finish()



print(f"Usando dispositivo: {DEVICE}")
experimentos = [
    {"lr":1e-3, "wd":1e-4, "batch":8, "epochs":15},
    {"lr":5e-4, "wd":1e-4, "batch":8, "epochs":15},
    {"lr":1e-3, "wd":1e-5, "batch":8, "epochs":20},
    {"lr":5e-4, "wd":1e-3, "batch":8, "epochs":20},
    {"lr":1e-4, "wd":1e-4, "batch":8, "epochs":25},
]
run_experiment("base", ["spectrograms"], experimentos)
run_experiment("augmented", ["spectrograms_augmented"], experimentos)


Usando dispositivo: cuda

=== Dataset: base ===

Ejecutando experimento 1/5 ...


wandb: Currently logged in as: javialroro (javialroro-tecnologico-de-costa-rica) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[01/15] Training Loss=3.7106 Training Acc=0.0600 | Validation Loss=3.4493 Validation Acc=0.0800 | F1=0.0252 Precision=0.0178 Recall=0.0712
[02/15] Training Loss=3.3588 Training Acc=0.0875 | Validation Loss=3.4210 Validation Acc=0.1050 | F1=0.0707 Precision=0.0671 Recall=0.1257
[03/15] Training Loss=3.2059 Training Acc=0.1056 | Validation Loss=3.1839 Validation Acc=0.1500 | F1=0.1226 Precision=0.1519 Recall=0.1617
[04/15] Training Loss=3.0056 Training Acc=0.1569 | Validation Loss=3.0255 Validation Acc=0.1700 | F1=0.1325 Precision=0.1316 Recall=0.1984
[05/15] Training Loss=2.7869 Training Acc=0.2094 | Validation Loss=2.7865 Validation Acc=0.2300 | F1=0.1541 Precision=0.1398 Recall=0.2406
[06/15] Training Loss=2.6014 Training Acc=0.2537 | Validation Loss=2.7410 Validation Acc=0.2950 | F1=0.2411 Precision=0.2240 Recall=0.3255
[07/15] Training Loss=2.3894 Training Acc=0.3113 | Validation Loss=2.4119 Validation Acc=0.3350 | F1=0.3005 Precision=0.3228 Recall=0.3710
[08/15] Training Loss=2.290

/home/javialroro/miniconda3/envs/ml/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/javialroro/miniconda3/envs/ml/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
f1,▁▂▂▂▃▄▅▄▅▆▇▇████
precision,▁▂▃▂▂▃▅▅▅▆▇▇▇▇██
recall,▁▂▂▃▃▄▅▅▆▆▇█████
test_acc,▁
train_acc,▁▁▂▂▃▃▄▄▅▆▆▇▇██
train_loss,█▇▆▆▅▅▄▄▃▃▂▂▁▁▁
val_acc,▁▁▂▂▃▄▅▄▅▆▇████
val_loss,██▇▆▆▅▄▄▃▃▂▂▁▁▁
epoch,15
f1,0.55651


...

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
f1,▁▂▂▄▄▃▄▄▅▄▆▆▆▆▇▇▇████
precision,▁▂▃▄▄▄▅▅▆▅▆▆▇▇▇▇▇████
recall,▁▂▂▄▄▃▄▄▅▄▆▆▆▆▇▇▇████
test_acc,▁
train_acc,▁▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇████
train_loss,█▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁
val_acc,▁▁▂▃▄▃▄▄▅▄▆▅▆▆▇▇▇███
val_loss,██▇▅▅▅▅▄▃▅▃▃▂▂▂▁▁▁▁▁
epoch,20
f1,0.87375



Ejecutando experimento 5/5 ...


[01/25] Training Loss=3.2069 Training Acc=0.1573 | Validation Loss=2.6549 Validation Acc=0.2633 | F1=0.2045 Precision=0.2425 Recall=0.2684
[02/25] Training Loss=2.5263 Training Acc=0.3067 | Validation Loss=2.2483 Validation Acc=0.3850 | F1=0.3433 Precision=0.4254 Recall=0.3813
[03/25] Training Loss=2.1016 Training Acc=0.4190 | Validation Loss=1.7374 Validation Acc=0.5300 | F1=0.4989 Precision=0.5630 Recall=0.5388
[04/25] Training Loss=1.8277 Training Acc=0.5017 | Validation Loss=1.6413 Validation Acc=0.5267 | F1=0.5174 Precision=0.6211 Recall=0.5304
[05/25] Training Loss=1.6013 Training Acc=0.5567 | Validation Loss=1.5146 Validation Acc=0.5833 | F1=0.5656 Precision=0.6486 Recall=0.5795
[06/25] Training Loss=1.3956 Training Acc=0.6117 | Validation Loss=1.3780 Validation Acc=0.5867 | F1=0.5679 Precision=0.6606 Recall=0.5889
[07/25] Training Loss=1.2476 Training Acc=0.6540 | Validation Loss=1.1763 Validation Acc=0.6550 | F1=0.6310 Precision=0.6797 Recall=0.6524
[08/25] Training Loss=1.086

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
f1,▁▂▄▄▄▄▅▅▆▆▇▇▇▇▇▇▇█████████
precision,▁▃▄▅▅▅▅▆▆▆▇▇▇▇▇▇▇█████████
recall,▁▂▄▄▄▄▅▅▆▆▇▇▆▇▇▇▇█████████
test_acc,▁
train_acc,▁▂▃▄▄▅▅▆▆▆▆▇▇▇▇▇█████████
train_loss,█▆▅▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_acc,▁▂▄▄▄▄▅▅▆▆▆▇▆▇▇▇▇████████
val_loss,█▇▅▅▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch,25
f1,0.96582



✅ Entrenamientos completados. Revisa 'runs_resnet/resnet_summary.csv' y tu panel en WandB.
